### 필수과제1. 임계값 조절하여 분류 테스트 변화 확인해  보기
- 임계값을 3개정도 (원하는 값으로) 잡고 각각의 평가 지표 변화 확인해 보기!

In [35]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import Binarizer
from sklearn.model_selection import train_test_split                  
from sklearn.linear_model import LogisticRegression      
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix            

In [24]:
import seaborn as sns
df = sns.load_dataset('titanic')              # seabron에서 제공하는 titanic 데이터 불러오기

In [14]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [25]:
df_tt=df[['survived','pclass','age','sibsp','parch','fare']] # 수치형 데이터만 임시로 추출
df_tt_x = df_tt[['pclass','sibsp','parch','fare']]           # 데이터(X)로 쓸 열들만 추출
df_tt_y = df_tt['survived']                                  # target(y)로 쓸 열만 추출

In [41]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
   
    print("오차 행렬")
    print(confusion)
    print(f"정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}")

In [61]:
X_train, X_test, y_train, y_test = train_test_split(df_tt_x, df_tt_y, test_size=0.2, random_state=1017)

# LogisticRegression에 모델 fit
lr_clf = LogisticRegression()
lr_clf.fit(X_train,y_train)

LogisticRegression()

In [48]:
lr_pred_proba = lr_clf.predict_proba(X_test)    # predict_proba() - Negative, Positive 확률 확인 가능
lr_pred = lr_clf.predict(X_test).reshape(-1,1)  

temp = pd.DataFrame( np.concatenate([lr_pred_proba,lr_pred], axis=1) )
temp.columns = ["Negative P", "Positive P", "predict"]
temp.head()

,Negative P,Positive P,predict
0,0.761513,0.238487,0.0
1,0.780028,0.219972,0.0
2,0.761540,0.238460,0.0
3,0.634809,0.365191,0.0
4,0.734996,0.265004,0.0


In [62]:
threshold = [0.3, 0.5, 0.7]
for thr in threshold:
  binarizer = Binarizer(threshold = thr)
  # positive 인 부분만을 가져와 threshold 지정한 거에 대입 후 fit, transform
  lr_pred_proba_po = lr_pred_proba[:,1].reshape(-1,1)
  my_pred = binarizer.fit_transform(lr_pred_proba_po)

  print("######## threshold =", thr, '일 때 ########')
  get_clf_eval(y_test,my_pred)
  print()

######## threshold = 0.3 일 때 ########
오차 행렬
[[69 38]
 [24 48]]
정확도: 0.6536, 정밀도: 0.5581, 재현율: 0.6667

######## threshold = 0.5 일 때 ########
오차 행렬
[[90 17]
 [38 34]]
정확도: 0.6927, 정밀도: 0.6667, 재현율: 0.4722

######## threshold = 0.7 일 때 ########
오차 행렬
[[104   3]
 [ 67   5]]
정확도: 0.6089, 정밀도: 0.6250, 재현율: 0.0694



- 재현율(Recall)과 정밀도(Precision)
  - 재현율(Recall) : True Positives / ( True Positives + False Negatives ) -> 실제 positive 데이터를 Negative로 잘못 판단하면 문제가 될 때
  - 정밀도(Precision) : True Positives / ( True Positives + False Positives ) -> 실제 negative 데이터를 positive로 잘못 판단하면 문제가 될 때



- Binarizer를 통해 threshold를 지정해 주지 않는 경우 기본적으로 threshold는 0.5로 설정이 된다.
  - threshold를 0.3으로 기준치보다 낮출 경우 정확도, 정밀도는 감소하고 재현율은 증가했다
  - threshold를 0.7로 기준치보다 높인 경우 정확도, 재현율은 감소하고 정밀도는 증가했다

- threshold가 0.3으로 낮아질 경우 
  - 기존 0.5보다 높으면 1, 낮으면 0 -> 0.3보다 높으면 1, 낮으면 0
  - positive(1)의 데이터 수가 감소하고, negative(0)의 데이터 수가 증가하게 된다 (TP,FP 감소 / TN,FN 증가)
  - 평가 결과 정밀도는 감소하고, 재현율은 증가한다

- threshold가 0.7로 높아질 경우 
  - 기존 0.5보다 높으면 1, 낮으면 0 -> 0.7보다 높으면 1, 낮으면 0
  - positive(1)의 데이터 수가 증가하고, negative(0)의 데이터 수가 감소하게 된다 (TP,FP 증가 / TN,FN 감소)
  - 평과 결과 정밀도는 감소하고, 재현율은 감소한다
